In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ~/input.txt

--2024-10-03 13:38:37--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘/Users/josemiguelvilchesfierro/input.txt’

/Users/josemiguelvi 100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-10-03 13:38:38 (9.91 MB/s) - ‘/Users/josemiguelvilchesfierro/input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('/Users/josemiguelvilchesfierro/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
list_ch = sorted(list(set(text)))
vocab_size = len(list_ch)
print("number of unique characters: ", len(list_ch))
print(list_ch)

number of unique characters:  65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
stoi = {ch:i for i,ch in enumerate(list_ch)}
itos = {i:ch for i,ch in enumerate(list_ch)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda h: ''.join([itos[i] for i in h])

encoded = encode('hello World')
decoded = decode(encode('hello World'))
print(encoded)
print(decoded)

[46, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
hello World


In [7]:
import torch
#Encode dataset
text_encode = torch.tensor(encode(text), dtype=torch.long)

In [8]:
#Create a dataset
n = int(len(text_encode) * 0.9)
train = text_encode[:n]
valid = text_encode[n:]

In [9]:
# x and y
block_size = 8
X = train[:block_size]
Y = train[1:block_size +1]
print(X)
print(Y)

for i in range(block_size):
    context = X[:i+1]
    target = Y[i]
    print(f"When X --> {context} then Y --> {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])
When X --> tensor([18]) then Y --> 47
When X --> tensor([18, 47]) then Y --> 56
When X --> tensor([18, 47, 56]) then Y --> 57
When X --> tensor([18, 47, 56, 57]) then Y --> 58
When X --> tensor([18, 47, 56, 57, 58]) then Y --> 1
When X --> tensor([18, 47, 56, 57, 58,  1]) then Y --> 15
When X --> tensor([18, 47, 56, 57, 58,  1, 15]) then Y --> 47
When X --> tensor([18, 47, 56, 57, 58,  1, 15, 47]) then Y --> 58


In [120]:
torch.manual_seed(1337)
batch_size = 4
block_size = 16
def get_batch(split):
    data = train if split == 'train' else valid
    ix = torch.randint(len(data)- block_size, (batch_size,))
    X = torch.stack([data[   i : i+block_size   ] for i in ix])
    Y = torch.stack([data[i + 1: i+block_size +1] for i in ix])
    return X, Y

Xb, Yb = get_batch('train')
print(f"{Xb.shape=}")
print(f"{Yb.shape=}")
print("-----------------")

for i in range(batch_size):
    for j in range(block_size):
        context = Xb[i, :j+1]
        target = Yb[i, j]
        print(f"When X --> {context} then Y --> {target}")

Xb.shape=torch.Size([4, 16])
Yb.shape=torch.Size([4, 16])
-----------------
When X --> tensor([21]) then Y --> 27
When X --> tensor([21, 27]) then Y --> 24
When X --> tensor([21, 27, 24]) then Y --> 13
When X --> tensor([21, 27, 24, 13]) then Y --> 26
When X --> tensor([21, 27, 24, 13, 26]) then Y --> 33
When X --> tensor([21, 27, 24, 13, 26, 33]) then Y --> 31
When X --> tensor([21, 27, 24, 13, 26, 33, 31]) then Y --> 10
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10]) then Y --> 0
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10,  0]) then Y --> 32
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10,  0, 32]) then Y --> 59
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10,  0, 32, 59]) then Y --> 57
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10,  0, 32, 59, 57]) then Y --> 46
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10,  0, 32, 59, 57, 46]) then Y --> 6
When X --> tensor([21, 27, 24, 13, 26, 33, 31, 10,  0, 32, 59, 57, 46,  6]) then Y --> 1
When X --> tensor([21, 27, 24, 

In [121]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguajeModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.embedding_tokken = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets= None):        
        #Embedding tokens and targets in a 3d tensor
        logits = self.embedding_tokken(idx) # B T C [4x 8 x 65]

        #Reshape the tensor to 2d for cross entropy
        if targets is None:
            loss = None
        else:
            B , T, C = logits.shape
            logits = logits.view(B*T , C) #[32 x 65]
            targets = targets.view(B*T) #[32]
            #Calculate loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the prediction
            logits, loss = self(idx) # [B x T x C] because targets is None [4 x 8 x 65]
            #get the last token
            logits = logits[:, -1, :] # Becomes [B x C] [4 x 65]
            #softmax
            probs = F.softmax(logits, dim=1) 
            #sample
            new_token = torch.multinomial(probs, 1)
            #append to the sequence
            idx = torch.cat([idx, new_token], dim=1) # B x T+1   

        return idx

model = BigramLanguajeModel(vocab_size)
model.generate(Xb, 10)

tensor([[21, 27, 24, 13, 26, 33, 31, 10,  0, 32, 59, 57, 46,  6,  1, 58, 31, 56,
         12, 55, 28,  7, 29, 35, 49, 58],
        [53, 59, 57,  1, 51, 43, 52,  0, 13, 56, 43,  1, 39, 58,  1, 58, 31,  0,
         60, 60, 47, 37, 16, 17, 57, 62],
        [50,  6,  1, 57, 47, 56,  8,  1, 18, 39, 56, 43,  1, 63, 53, 59, 16,  4,
         55, 29, 16, 15, 34,  4, 53, 48],
        [58, 46,  1, 57, 53,  6,  1, 46, 53, 50, 63,  1, 57, 47, 56, 11,  4, 29,
         10, 17, 22, 46,  3, 44, 30, 18]])

In [122]:
optimier = torch.optim.Adam(model.parameters(), lr=1e-3)

In [124]:
batch_size = 64
for i in range(5000):

    if i % 100 == 0:
        losses = estimate_loss()
        print(f"Step {i} -- Train Loss: {losses['train']:.2f} -- Valid Loss: {losses['valid']:.2f}")

    #forward step
    xb, yb = get_batch('train')
    logits, loss = model.forward(Xb, Yb)

    #backward step
    optimier.zero_grad(set_to_none=True)
    loss.backward()
    optimier.step()



loss: 2.947575569152832
loss: 2.7999722957611084
loss: 2.659029245376587
loss: 2.524958848953247
loss: 2.3979098796844482
loss: 2.277965784072876
loss: 2.165147304534912
loss: 2.0594098567962646
loss: 1.960646390914917
loss: 1.868687629699707
loss: 1.7833110094070435
loss: 1.7042453289031982
loss: 1.6311838626861572
loss: 1.5637930631637573
loss: 1.5017261505126953
loss: 1.4446297883987427
loss: 1.3921564817428589
loss: 1.3439686298370361
loss: 1.299744725227356
loss: 1.2591819763183594
loss: 1.2219980955123901
loss: 1.187931776046753
loss: 1.1567413806915283
loss: 1.1282044649124146
loss: 1.1021153926849365
loss: 1.0782831907272339
loss: 1.0565308332443237
loss: 1.0366913080215454
loss: 1.0186084508895874
loss: 1.0021346807479858
loss: 0.9871307611465454
loss: 0.9734655618667603
loss: 0.961016833782196
loss: 0.9496700167655945
loss: 0.9393194913864136
loss: 0.9298681020736694
loss: 0.9212271571159363
loss: 0.9133164882659912
loss: 0.9060633182525635
loss: 0.899402916431427
loss: 0.893

In [126]:
idx = torch.zeros((1,1), dtype=torch.long)
generate = model.generate(idx, 400)[0].tolist()
print(decode(generate))


ANUS:
Ty sousire ar; sol, -wNUS:CDn
Ar. men
Are d, C-bYLAr. sh tu yousir; s soly Fat men
ANUS:
T'QjQ'BS:
Are hush yous Far. Ml, h, thol, thou h yol, tu me so, y men
Tu!P!zPe sho, ar; ar. mY-jKuso&xqxZQyousous t aren
Tush at tth siren
Tush, sire yol, t sh h men
Tu h, me me t yo, Fare ath, FathRl, Fare tus at men
Ar. sB$KpUS:
ANt atholyHqMOLAr; shol, you ho, sous t Faren
ANUS:
AV; at t sir. ywo, th 
